In [0]:
# Intro to Data Science, Fall 2019 @ CCNY
# CSC-599.70-Course-Project
# Team Member & Name: RentAdvisor (3-Member)

# HASIBUL ISLAM
# DZHONIBEK PARMANKULOV
# ABDUR RAFEY

In [0]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)


In [3]:
train_df.sample(5)

,addr_unit,building_id,bedrooms,bathrooms,size_sqft,created_at,addr_street,addr_city,addr_zip,addr_lat,addr_lon,bin,bbl,floor_count,year_built,min_to_subway,has_doorman,has_elevator,has_fireplace,has_dishwasher,is_furnished,has_gym,allows_pets,has_washer_dryer,has_garage,has_roofdeck,has_concierge,has_pool,has_garden,has_childrens_playroom,rent,no_fee,description,neighborhood,borough,unit,floornumber,line
rental_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7415533,#2D,370183,2,1.0,750.0,2018-07-27 12:14:19,195 DRIGGS AVENUE,Brooklyn,11222,40.722672,-73.946134,3067573.0,3027000018,5.0,1905.0,2.4833,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2395,0,***Heart of Greenpoint!***\nJust hit the marke...,Greenpoint,Brooklyn,2D,2.0,D
7410316,#3,201967,2,1.0,1000.0,2018-07-25 21:42:46,83 ST MARK'S PLACE,Brooklyn,11217,40.681794,-73.979276,3018574.0,3009320075,3.0,1910.0,2.2833,0,0,0,0,0,0,1,1,0,0,0,0,1,0,2999,0,PARK SLOPE NORTH\r\nMinutes Atlantic Terminal ...,Park Slope,Brooklyn,3,3.0,NaN
7329139,#515,460,0,1.0,391.0,2018-07-03 11:13:09,63 WALL STREET,New York,10005,40.705799,-74.008698,1000828.0,1000270009,37.0,2004.0,0.9167,1,1,0,1,0,1,1,1,0,0,1,0,0,0,2755,1,Now and for a limited time we are offering: \n...,Financial District,Manhattan,515,5.0,15
7135096,#27L,55885,2,2.0,1021.0,2018-05-01 16:09:16,30 WEST 63 STREET,New York,10023,40.770802,-73.981598,1027463.0,1011157503,33.0,1979.0,3.4333,1,1,0,1,0,1,1,0,0,1,1,1,0,1,6895,1,Incentive: 1 month OP to broker & 1 month free...,Upper West Side,Manhattan,27L,27.0,L
7265446,#5A,1864513,2,1.0,800.0,2018-06-12 14:19:43,135-23 82 AVENUE,Jamaica,11435,40.714842,-73.822076,4206575.0,4096690030,6.0,2009.0,6.6333,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1995,0,DELICIOUS LUXURY NEW CONSTRUCTION 2BR NEAR E/F...,Briarwood,Queens,5A,5.0,A


In [0]:
# extract usable features
feature_cols = [
    'bedrooms', 'year_built', 'bathrooms', 'min_to_subway', 
    'size_sqft', 'no_fee', 'has_doorman','addr_zip',
    'floor_count',#'has_elevator',#'has_dishwasher',	#'is_furnished',	
    'has_gym','allows_pets',#'has_garage',
    'has_concierge',#'has_pool',#'has_garden',	
    #the reason of commenting out certain feature is because they are increasing mean squared error
]
train_features = train_df[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train_df['rent']

In [127]:
#using linear regression method
lreg = LinearRegression()
lreg.fit(train_features, train_target)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [128]:
test_features = test_df[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test_df['predicted'] = lreg.predict(test_features)
print("Prediction using train data against test 1:")

print("Mean Squared Error for Test1 :",mean_squared_error(test_df['rent'], test_df['predicted']))

Prediction using train data against test 1:
Mean Squared Error for Test1 : 3313817.143868871


In [129]:
#joining training data and test1 data to incredase total training data
master_df = train_df.append(test_df, sort=False)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']
#linear regression
lreg.fit(master_features, master_target)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [132]:
submit1_features = submit1_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit1_df['rent'] = lreg.predict(submit1_features)
submit1_df['rent'].to_csv('test2.csv', header=True)
#these part has to be changed
submit1_df['median_rent'] = np.ones(submit1_df['rent'].shape) * master_target.median()
print("Prediction of rent for test2")
print("Mean Squared error for test 2: ",mean_squared_error(submit1_df['rent'], submit1_df['median_rent']))

Prediction of rent for test2
Mean Squared error for test 2:  5058501.210167723


In [131]:
submit1_df.sample(5)

,addr_unit,building_id,bedrooms,bathrooms,size_sqft,created_at,addr_street,addr_city,addr_zip,addr_lat,addr_lon,bin,bbl,floor_count,year_built,min_to_subway,has_doorman,has_elevator,has_fireplace,has_dishwasher,is_furnished,has_gym,allows_pets,has_washer_dryer,has_garage,has_roofdeck,has_concierge,has_pool,has_garden,has_childrens_playroom,rent,no_fee,description,neighborhood,borough,unit,floornumber,line,median_rent
rental_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7525654,#25B,77122,3,3.0,1653.0,2018-08-31 16:03:36,200 EAST 69 STREET,New York,10065,40.767300,-73.961800,1043902.0,1014237501,57.0,1991.0,1.4667,1,1,0,0,0,1,1,1,1,0,1,0,0,0,10378.415814,0,JUST REDUCED! BEST PRICED 3BR W/BALCONY! Great...,Upper East Side,Manhattan,25B,25.0,B,2900.0
7484698,#3L,397198,3,1.0,850.0,2018-08-16 21:02:27,173 IRVING AVENUE,Brooklyn,11237,40.702329,-73.919703,3074225.0,3032590007,3.0,1931.0,2.3500,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2727.960493,0,"This is an amazing, beautifully renovated apar...",Bushwick,Brooklyn,3L,3.0,L,2900.0
7495918,#4G,57391,1,1.0,675.0,2018-08-21 10:54:43,130 WEST 67 STREET,New York,10023,40.774799,-73.983299,1028847.0,1011380053,25.0,1975.0,1.3000,1,1,0,1,0,0,0,0,0,1,0,0,1,0,3452.527929,0,** NET EFFECTIVE RENT - ONE FREE MONTH ON A 13...,Upper West Side,Manhattan,4G,4.0,G,2900.0
7505905,#3,18352870,2,1.0,0.0,2018-08-23 19:02:42,22-29 79 STREET,East Elmhurst,11370,40.767799,-73.889999,4449341.0,4010117501,0.0,NaN,21.3667,0,0,0,1,0,0,0,1,0,0,0,0,0,0,47.439895,0,"Luxury Finishes!!!!!\r\nGranite Counter Tops, ...",Astoria,Queens,3,3.0,NaN,2900.0
7481782,#3,48274,1,1.0,0.0,2018-08-16 11:11:29,238 EAST 33 STREET,New York,10016,40.744301,-73.977402,1078802.0,1009130044,5.0,1880.0,3.8000,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1764.391610,0,THE ONLY 1 BEDROOM IN KIPS BAY AT THIS PRICE T...,Midtown East,Manhattan,3,3.0,NaN,2900.0
